In [1]:
!pip install -q autogluon.tabular
!pip install -q ray

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
aiobotocore 2.13.1 requires aiohttp<4.0.0,>=3.9.2, but you have aiohttp 3.9.1 which is incompatible.
aiobotocore 2.13.1 requires botocore<1.34.132,>=1.34.70, but you have botocore 1.29.165 which is incompatible.
spaghetti 1.7.6 requires shapely>=2.0.1, but you have shapely 1.8.5.post1 which is incompatible.
spopt 0.6.1 requires shapely>=2.0.1, but you have shapely 1.8.5.post1 which is incompatible.


In [2]:
from autogluon.tabular import TabularPredictor
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import matthews_corrcoef
from sklearn.base import clone
import pandas as pd
import numpy as np
import warnings
import pickle
import shutil
import gc

warnings.filterwarnings('ignore')

In [3]:
SEED = 6
N_FOLDS = 5
TARGET = 'class'
TIME_LIMIT = 3600 * 7

In [4]:
train = pd.read_csv('/kaggle/input/playground-series-s4e8/train.csv', index_col='id')
test = pd.read_csv('/kaggle/input/playground-series-s4e8/test.csv', index_col='id')

In [5]:
def save_pred_probs(pred_probs, cv_score, name):
    with open(f'autogluon_{name}_pred_probs_{cv_score:.6f}.pkl', 'wb') as f:
        pickle.dump(pred_probs, f)

def save_submission(test_pred_probs, score):
    sub = pd.read_csv('/kaggle/input/playground-series-s4e8/sample_submission.csv')
    sub[TARGET] = np.argmax(test_pred_probs, axis=1)
    sub[TARGET] = sub[TARGET].map({0: 'e', 1: 'p'})
    sub.to_csv(f'sub_autogluon_{score:.6f}.csv', index=False)

In [6]:
scores = []    
oof_pred_probs = np.zeros((train.shape[0], len(np.unique(train[TARGET]))))
test_pred_probs = np.zeros((test.shape[0], len(np.unique(train[TARGET]))))

skf = StratifiedKFold(n_splits=N_FOLDS, random_state=SEED, shuffle=True)
split = skf.split(train.drop(columns=TARGET), train[TARGET])
for fold_idx, (train_idx, val_idx) in enumerate(split):
    _train, _val = train.iloc[train_idx], train.iloc[val_idx]   
    
    predictor = TabularPredictor(
        label=TARGET,
        eval_metric='mcc',
        problem_type='binary',
        verbosity=2
    )
    
    predictor.fit(
        train_data=_train,
        time_limit=TIME_LIMIT // N_FOLDS,
        presets='best_quality',
        excluded_model_types=['KNN'],
        ag_args_fit={
            'num_gpus': 1, 
            'stopping_metric': 'log_loss'
        }
    )

    y_pred_probs = predictor.predict_proba(_val)
    oof_pred_probs[val_idx] = y_pred_probs            

    temp_test_pred_probs = predictor.predict_proba(test)
    test_pred_probs += temp_test_pred_probs / N_FOLDS

    score = matthews_corrcoef(_val[TARGET].map({'e': 0, 'p': 1}), np.argmax(y_pred_probs, axis=1))
    scores.append(score)

    del predictor, _train, _val, y_pred_probs, temp_test_pred_probs
    gc.collect()

No path specified. Models will be saved in: "AutogluonModels/ag-20240810_180613"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.10.13
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Thu Jun 27 20:43:36 UTC 2024
CPU Count:          4
Memory Avail:       28.14 GB / 31.36 GB (89.7%)
Disk Space Avail:   19.50 GB / 19.52 GB (99.9%)
Presets specified: ['best_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
	This is used to identify the optimal `num_stack_levels` value. Copies of AutoGluon will be fit on subsets o

[1000]	valid_set's binary_logloss: 0.0358903


	0.9842	 = Validation score   (mcc)
	719.63s	 = Training   runtime
	82.75s	 = Validation runtime
Fitting model: LightGBM_BAG_L1 ... Training model for up to 8.72s of the 417.53s of remaining time.
	Fitting 8 child models (S1F1 - S1F8) | Fitting with SequentialLocalFoldFittingStrategy
	Training S1F1 with GPU, note that this may negatively impact model quality compared to CPU training.
	Ran out of time, early stopping on iteration 1. Best iteration is:
	[1]	valid_set's binary_logloss: 0.654943
	Time limit exceeded... Skipping LightGBM_BAG_L1.
Fitting model: RandomForestGini_BAG_L1 ... Training model for up to 4.41s of the 413.22s of remaining time.
	Time limit exceeded... Skipping RandomForestGini_BAG_L1.
Fitting model: WeightedEnsemble_L2 ... Training model for up to 360.0s of the 399.83s of remaining time.
	Ensemble Weights: {'LightGBMXT_BAG_L1': 1.0}
	0.9842	 = Validation score   (mcc)
	0.26s	 = Training   runtime
	0.34s	 = Validation runtime
Excluded models: ['KNN'] (Specified by `ex

[1000]	valid_set's binary_logloss: 0.0366087
[2000]	valid_set's binary_logloss: 0.0362303
[3000]	valid_set's binary_logloss: 0.0361472
[4000]	valid_set's binary_logloss: 0.0361371


	Ran out of time, early stopping on iteration 4074. Best iteration is:
	[3245]	valid_set's binary_logloss: 0.036126
	Training S1F2 with GPU, note that this may negatively impact model quality compared to CPU training.


...

	Ran out of time, early stopping on iteration 1348. Best iteration is:
	[1348]	valid_set's binary_logloss: 0.0361599
	Training S1F7 with GPU, note that this may negatively impact model quality compared to CPU training.


[1000]	valid_set's binary_logloss: 0.0376473


	Ran out of time, early stopping on iteration 1370. Best iteration is:
	[1369]	valid_set's binary_logloss: 0.0375542
	Training S1F8 with GPU, note that this may negatively impact model quality compared to CPU training.


[1000]	valid_set's binary_logloss: 0.0358751


	Ran out of time, early stopping on iteration 1553. Best iteration is:
	[1551]	valid_set's binary_logloss: 0.0356896
	0.9846	 = Validation score   (mcc)
	1051.68s	 = Training   runtime
	124.45s	 = Validation runtime
Fitting model: LightGBM_BAG_L2 ... Training model for up to 17.92s of the 17.82s of remaining time.
	Fitting 8 child models (S1F1 - S1F8) | Fitting with SequentialLocalFoldFittingStrategy
	Training S1F1 with GPU, note that this may negatively impact model quality compared to CPU training.
	Ran out of time, early stopping on iteration 1. Best iteration is:
	[1]	valid_set's binary_logloss: 0.641358
	Time limit exceeded... Skipping LightGBM_BAG_L2.
Fitting model: RandomForestGini_BAG_L2 ... Training model for up to 12.74s of the 12.65s of remaining time.
	Time limit exceeded... Skipping RandomForestGini_BAG_L2.
Fitting model: WeightedEnsemble_L3 ... Training model for up to 360.0s of the -4.74s of remaining time.
	Ensemble Weights: {'LightGBMXT_BAG_L1': 1.0}
	0.9847	 = Validat

In [7]:
for i in range(N_FOLDS):
    print(f'Fold {i + 1} - MCC: {scores[i]:.6f}')
    
print(f'\n--- Average MCC: {np.mean(scores):.6f} ± {np.std(scores):.6f}')

Fold 1 - MCC: 0.984637
Fold 2 - MCC: 0.985046
Fold 3 - MCC: 0.984650
Fold 4 - MCC: 0.984817
Fold 5 - MCC: 0.984847

--- Average MCC: 0.984799 ± 0.000150


In [8]:
save_pred_probs(oof_pred_probs, np.mean(scores), 'oof')
save_pred_probs(test_pred_probs, np.mean(scores), 'test')
save_submission(test_pred_probs, np.mean(scores)) 

In [9]:
shutil.rmtree("AutogluonModels")